In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import matplotlib.ticker as ticker

# Function to read and filter data from Excel
def filter_event_data(input_file, event_type='Parameters', end_date_str='5/30/2023', months_offset=-4):
    # Read the Excel file
    df = pd.read_excel(input_file)

    # Filter rows where 'Event' is not 'Parameters'
    df = df[df['Event'] == event_type]

    # Convert 'Date' column to datetime format
    df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')

    # Calculate start date
    end_date = pd.to_datetime(end_date_str)
    start_date = end_date + pd.DateOffset(months=months_offset)

    # Filter the DataFrame for the specified date range
    df_filtered = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    
    return df_filtered

# Function to save DataFrame to Excel
def save_to_excel(df, output_file):
    df.to_excel(output_file, index=False)

# Function to plot grouped data with multiple y-axes
def plot_grouped_data(df, group_dict, chart_title, output_file=None, line_width=1.25):
    fig, ax = plt.subplots(figsize=(18, 8))
    fig.subplots_adjust(right=0.75)

    # Initialize axes
    axes = [ax]

    # Create additional y-axes
    for _ in range(len(group_dict) - 1):
        axes.append(ax.twinx())
        axes[-1].spines.right.set_position(("axes", 1 + 0.07 * len(axes)))

    # Generate distinct colors for all lines
    all_columns = [col for columns in group_dict.values() for col in columns]
    colors = sns.color_palette("tab10", len(all_columns))

    color_index = 0  # Initialize a color index

    # Plot each group of data
    for i, (label, columns) in enumerate(group_dict.items()):
        for col in columns:
            if col in df.columns:
                sns.lineplot(x=df.index, y=df[col], ax=axes[i], label=col, color=colors[color_index], linewidth=line_width)
                color_index += 1

        # Set y-axis label and limits
        axes[i].set_ylabel(label)
        axes[i].tick_params(axis='y', labelcolor=colors[color_index - 1])
        axes[i].yaxis.label.set_color(colors[color_index - 1])

        # Manually set y-axis limits based on the label
        set_y_limits(label, axes[i])

        # Set the y-ticks to make them evenly spaced
        axes[i].yaxis.set_major_locator(ticker.MaxNLocator(integer=True))

        # Enable grid only on the primary y-axis
        if i == 0:
            axes[i].grid(True, which='major', linestyle='-', linewidth=0.5)
        else:
            axes[i].grid(False)  # Disable grid for additional y-axes

    # Format x-axis
    ax.set_xlabel("Date")
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%Y'))
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    fig.autofmt_xdate()

    # Set the title
    ax.set_title(chart_title, loc='center', pad=50)

    # Handle the legend
    lines, labels = [], []
    for axis in axes:
        line, label = axis.get_legend_handles_labels()
        lines += line
        labels += label
    fig.legend(lines, labels, loc='upper right', bbox_to_anchor=(0.64, 0.92), ncol=len(labels), frameon=False)

    # Save the plot to a file if output_file is provided
    if output_file:
        plt.savefig(output_file, bbox_inches='tight')

    # Show the plot
    plt.show()

# Function to set y-axis limits based on label
def set_y_limits(label, axis):
    limits = {
        'Temperature (°C)': [-10, 100],
        'Pressure (PSI)': [1300, 8000],
        'Frequency (Hz)': [0, 80],
        'Motor Load (%)': [0, 80],
        'Voltage (V)': [370, 400],
        'Current (A)': [0, 160],
        'Resistance (kOhm)': [-1000, 10000],
        'Imbalance (%)': [0, 3]
    }
    if label in limits:
        axis.set_ylim(limits[label])

# Main execution flow
if __name__ == '__main__':
    # Filter and save event data
    input_file = 'YRN-16.xlsx'
    df_filtered = filter_event_data(input_file)
    save_to_excel(df_filtered, 'YRN-16.a.xlsx')

    # Load data for plotting
    well_name = 'YRN-11.b'
    df_plot = pd.read_excel('YRN-11.b.xlsx')
    df_plot['Date'] = pd.to_datetime(df_plot['Date'])
    df_plot.set_index('Date', inplace=True)

    # Define the chart groupings
    chart1_groups = {
        'Temperature (°C)': ['Tin.. C'],
        'Pressure (PSI)': ['Pin.. PSI', 'Pout. PSI'],
        'Frequency (Hz)': ['Fout. Hz'],
        'Motor Load (%)': ['Motor load. %']
    }

    chart2_groups = {
        'Voltage (V)': ['Uin.AB, V'],
        'Current (A)': ['Iu. A'],
        'Resistance (kOhm)': ['Rins, kOhm'],
        'Imbalance (%)': ['Uin.imbal.. %', 'Imot imbal.. %']
    }

    # Plot charts
    plot_grouped_data(df_plot, chart1_groups, "Temperatures, Pressures, Frequency, and Motor Load", output_file=f"{well_name} Temperatures, Pressures, Frequency, and Motor Load.png")
    plot_grouped_data(df_plot, chart2_groups, "Voltage and Current", output_file=f"{well_name} Voltage and Current.png")
